# 这题我还没做对

In [97]:
import numpy as np
import pandas as pd
import gurobipy as gp
from math import log
from gurobipy import GRB
from decimal import Decimal

In [98]:
df = pd.read_csv("2.4.csv")

athletes = df.columns
sports = pd.Index(("Sport1", "Sport2", "Sport3", "Sport4"))
df.index = sports

df_long = pd.melt(
    df.reset_index(), id_vars="index", var_name="运动员", value_name="得分~概率"
)
df_long.columns = ["项目", "运动员", "得分~概率"]

expanded_data = []
for i, row in df_long.iterrows():
    for item in row["得分~概率"].split(" "):
        score, prob = map(Decimal, item.split("~"))
        expanded_data.append(
            {"项目": row["项目"], "运动员": row["运动员"], "得分": score, "概率": prob}
        )
expanded_df = pd.DataFrame(expanded_data)
# 计算最低得分和期望平均分
summary_stats = (
    expanded_df.groupby(["项目", "运动员"])
    .apply(
        lambda x: pd.Series(
            {"最低得分": x["得分"].min(), "期望平均分": (x["得分"] * x["概率"]).sum()}
        )
    )
    .reset_index()
)

/var/folders/n_/d82sr3dd4jgfqk4vk5sblcn80000gn/T/ipykernel_89198/2879913793.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [99]:
prob_combos, scores, probs = gp.multidict(
    {
        (row["项目"], row["运动员"], row["得分"]): [row["得分"], row["概率"]]
        for _, row in expanded_df.iterrows()
    }
)

In [100]:
m = gp.Model()

In [101]:
x = m.addVars(prob_combos, vtype=GRB.BINARY, name="x")
y = m.addVars(athletes, vtype=GRB.BINARY, name="y")

In [102]:
m.setObjective(x.prod(probs), sense=GRB.MAXIMIZE)

In [103]:
m.addConstr(y.sum() == 4, name="应有四人参加全能比赛")

<gurobi.Constr *Awaiting Model Update*>

In [104]:
m.addConstrs((x.sum(sport, "*") == 6 for sport in sports), name="每个项目最多出场六人")

{'Sport1': <gurobi.Constr *Awaiting Model Update*>,
 'Sport2': <gurobi.Constr *Awaiting Model Update*>,
 'Sport3': <gurobi.Constr *Awaiting Model Update*>,
 'Sport4': <gurobi.Constr *Awaiting Model Update*>}

In [105]:
m.addConstrs(
    (x.sum("*", athlete) <= 3 + y[athlete] for athlete in athletes),
    name="是否参加全能比赛上限约束",
)
m.addConstrs(
    (4 * y[athlete] <= x.sum("*", athlete) for athlete in athletes),
    name="是否参加全能比赛下限约束",
)

{'Athlete1': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete2': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete3': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete4': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete5': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete6': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete7': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete8': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete9': <gurobi.Constr *Awaiting Model Update*>,
 'Athlete10': <gurobi.Constr *Awaiting Model Update*>}

In [106]:
x["Sport1", "Athlete1", Decimal("8.4")]

<gurobi.Var *Awaiting Model Update*>

In [107]:
m.addConstrs(
    (x.sum(sport, athlete, "*") <= 1 for sport in sports for athlete in athletes),
    name="只可能是一种结果",
)

{('Sport1', 'Athlete1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport1', 'Athlete2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport1', 'Athlete3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport1', 'Athlete4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport1', 'Athlete5'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport1', 'Athlete6'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport1', 'Athlete7'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport1', 'Athlete8'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport1', 'Athlete9'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport1', 'Athlete10'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport2', 'Athlete1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport2', 'Athlete2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport2', 'Athlete3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport2', 'Athlete4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport2', 'Athlete5'): <gurobi.Constr *Awaiting Model Update*>,
 ('Sport2

In [108]:
m.setObjective(x.prod(probs), sense=GRB.MAXIMIZE)

In [109]:
m.addConstr(x.prod(scores) >= 236.2, name="团体总分约束")

<gurobi.Constr *Awaiting Model Update*>

In [110]:
m.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 66 rows, 170 columns and 830 nonzeros
Model fingerprint: 0xd3743691
Variable types: 0 continuous, 170 integer (170 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [5e-02, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 5 rows and 79 columns
Presolve time: 0.00s
Presolved: 61 rows, 91 columns, 421 nonzeros
Variable types: 0 continuous, 91 integer (91 binary)

Root relaxation: objective 6.700000e+00, 56 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.70000    0    5          -    6.70000      -     -    0s
H    0     0                       

In [111]:
print(f"Objective value: {m.ObjVal}")
print("Optimal solution:")
for i in m.getVars():
    if i.X > 1e-6:
        print(f"{i.VarName} = {i.X}", end="\n")

Objective value: 5.4
Optimal solution:
x[Sport1,Athlete1,9.5] = 1.0
x[Sport2,Athlete1,10] = 1.0
x[Sport3,Athlete1,9.8] = 1.0
x[Sport4,Athlete1,9.9] = 1.0
x[Sport3,Athlete2,10] = 1.0
x[Sport1,Athlete3,10] = 1.0
x[Sport4,Athlete3,9.8] = 1.0
x[Sport2,Athlete4,9.9] = 1.0
x[Sport3,Athlete4,9.7] = 1.0
x[Sport4,Athlete4,10] = 1.0
x[Sport1,Athlete6,9.7] = 1.0
x[Sport2,Athlete6,9.9] = 1.0
x[Sport1,Athlete7,10] = 1.0
x[Sport2,Athlete7,10] = 1.0
x[Sport3,Athlete7,9.3] = 1.0
x[Sport4,Athlete7,9.8] = 1.0
x[Sport1,Athlete8,10] = 1.0
x[Sport2,Athlete8,10] = 1.0
x[Sport3,Athlete8,9.9] = 1.0
x[Sport4,Athlete8,9.8] = 1.0
x[Sport1,Athlete9,9.5] = 1.0
x[Sport2,Athlete9,9.8] = 1.0
x[Sport3,Athlete9,10] = 1.0
x[Sport4,Athlete9,9.9] = 1.0
y[Athlete1] = 1.0
y[Athlete7] = 1.0
y[Athlete8] = 1.0
y[Athlete9] = 1.0
